In [ ]:
import numpy as np
import math
import pandas as pd
from quantopian.research import run_pipeline, returns, get_pricing
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import CustomFactor, Returns
from quantopian.pipeline.filters import QTradableStocksUS
from quantopian.pipeline.data import Fundamentals
from quantopian.pipeline.data.factset import Fundamentals as FstFundamentals
from quantopian.pipeline import factors, filters, classifiers
from quantopian.pipeline.data import factset
import alphalens
from sklearn import preprocessing
from scipy.stats.mstats import winsorize


sd = '2010-01-01'
ed = '2018-10-01'
QL = 80
WIN_LIMIT = 0

def signalize(df):
   return ((df.rank() - 0.5)/df.count()).replace(np.nan,0.5)


def preprocess(a):
    a = a.astype(np.float64)
    a[np.isinf(a)] = np.nan
    a = np.nan_to_num(a - np.nanmean(a))
    a = winsorize(a, limits=[WIN_LIMIT,WIN_LIMIT])
    
    return preprocessing.scale(a)


def transform(df, field, multiplier=1):
    return signalize(multiplier*df[field])

class MarketCap(CustomFactor):
    # Pre-declare inputs and window_length
    inputs = [USEquityPricing.close, Fundamentals.shares_outstanding]
    window_length = 1

    # Compute market cap value
    def compute(self, today, assets, out, close, shares):
        out[:] = close[-1] * shares[-1]

class LogMarketCap(CustomFactor):
    # Pre-declare inputs and window_length
    inputs = [USEquityPricing.close, Fundamentals.shares_outstanding]
    window_length = 1

    # Compute market cap value
    def compute(self, today, assets, out, close, shares):
        out[:] = np.log(close[-1] * shares[-1])

class Volatility(CustomFactor):  
    inputs = [USEquityPricing.close]  
    window_length = 20  
    def compute(self, today, assets, out, close):  
        # [0:-1] is needed to remove last close since diff is one element shorter  
        daily_returns = np.diff(close, axis = 0) / close[0:-1]  
        out[:] = daily_returns.std(axis = 0) * math.sqrt(252)

class Industry(CustomFactor):  
    inputs = [Fundamentals.morningstar_industry_code]  
    window_length = 1 
    def compute(self, today, assets, out, code):  
        out[:] = code[-1]

        
class IndustryGroup(CustomFactor):  
    inputs = [Fundamentals.morningstar_industry_group_code]  
    window_length = 1 
    def compute(self, today, assets, out, code):  
        out[:] = code[-1]


class Sector(CustomFactor):  
    inputs = [Fundamentals.morningstar_sector_code]  
    window_length = 1 
    def compute(self, today, assets, out, code):  
        out[:] = code[-1]


class Var_growth_mean(CustomFactor):    
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, var):  
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
        arr = np.diff(arr, axis=0)/arr[1:]
        mean = np.nanmean(arr, axis=0)
        out[:] = preprocess(mean)


class Var_growth_q0(CustomFactor):  
    window_length = QL
    window_safe = True
    
    def compute(self, today, assets, out, var):  
        gr = var[-1]/var[-QL] - 1.0
        out[:] = preprocess(gr)

        
class Var_growth_stability(CustomFactor):  
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, var):  
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
        arr = np.diff(arr, axis=0)/arr[1:]
        mean = np.nanmean(arr, axis=0)
        std = np.nanstd(arr, axis=0)
        out[:] = preprocess(mean/std)
        
class Var_to_TA_mean(CustomFactor):  
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, v, ta):  
        var  = v/ta
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
        mean = np.nanmean(arr, axis=0)
        out[:] = preprocess(mean)
        
        
class Var_to_TA_stability(CustomFactor):  
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, v, ta):  
        var  = v/ta
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
        mean = np.nanmean(arr, axis=0)
        std = np.nanstd(arr, axis=0)
        out[:] = preprocess(mean/std)
        
class Var_to_TL_mean(CustomFactor):  
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, v, ta):  
        var  = v/ta
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
        mean = np.nanmean(arr, axis=0)
        out[:] = preprocess(mean)
        
        
class Var_to_TL_stability(CustomFactor):  
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, v, ta):  
        var  = v/ta
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
        mean = np.nanmean(arr, axis=0)
        std = np.nanstd(arr, axis=0)
        out[:] = preprocess(mean/std)
        
class Var_to_TC_mean(CustomFactor):  
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, v, ta):  
        var  = v/ta
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
        mean = np.nanmean(arr, axis=0)
        out[:] = preprocess(mean)
        
    
class Var_to_TC_stability(CustomFactor):  
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, v, ta):  
        var  = v/ta
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
        mean = np.nanmean(arr, axis=0)
        std = np.nanstd(arr, axis=0)
        out[:] = preprocess(mean/std)

class Var_to_EV_mean(CustomFactor):  
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, v, ta):  
        var  = v/ta
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
        mean = np.nanmean(arr, axis=0)
        out[:] = preprocess(mean)

        
class Var_to_EV_stability(CustomFactor):  
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, v, ta):  
        var  = v/ta
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
        mean = np.nanmean(arr, axis=0)
        std = np.nanstd(arr, axis=0)
        out[:] = preprocess(mean/std)
        
class Var_to_TE_mean(CustomFactor):  
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, v, ta):  
        var  = v/ta
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
        mean = np.nanmean(arr, axis=0)
        out[:] = preprocess(mean)
        
        
class Var_to_TE_stability(CustomFactor):  
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, v, ta):  
        var  = v/ta
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
        mean = np.nanmean(arr, axis=0)
        std = np.nanstd(arr, axis=0)
        out[:] = preprocess(mean/std)
        
        
class Var_to_TR_mean(CustomFactor):  
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, v, ta):  
        var  = v/ta
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
        mean = np.nanmean(arr, axis=0)
        out[:] = preprocess(mean)
        
        
class Var_to_TR_stability(CustomFactor):  
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, v, ta):  
        var  = v/ta
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
        mean = np.nanmean(arr, axis=0)
        std = np.nanstd(arr, axis=0)
        out[:] = preprocess(mean/std)
        
        
class Var_to_TInv_mean(CustomFactor):  
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, v, ta):  
        var  = v/ta
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
        mean = np.nanmean(arr, axis=0)
        out[:] = preprocess(mean)

        
class Var_to_TInv_stability(CustomFactor):  
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, v, ta):  
        var  = v/ta
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
        mean = np.nanmean(arr, axis=0)
        std = np.nanstd(arr, axis=0)
        out[:] = preprocess(mean/std)
        

class Var_to_MCap_mean(CustomFactor):  
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, v, close, shares):  
        var  = v/(close*shares)
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
        mean = np.nanmean(arr, axis=0)
        out[:] = preprocess(mean)
    
    
class Var_to_MCap_stability(CustomFactor):  
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, v, close, shares):  
        var  = v/(close*shares)
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
        mean = np.nanmean(arr, axis=0)
        std = np.nanstd(arr, axis=0)
        out[:] = preprocess(mean/std)


base_universe = QTradableStocksUS()

mkt_cap = MarketCap(mask = base_universe)
log_mkt_cap = LogMarketCap(mask = base_universe)
vol = Volatility(mask = base_universe)

industry = Industry(mask = base_universe)
industryGroup = IndustryGroup(mask = base_universe)
sector = Sector(mask = base_universe)

#Factor - 1 
# [4,6,14,15,18,23]
# f1 = Var_growth_mean(inputs = [Fundamentals.trading_liabilities], mask = base_universe)
# f2 = Var_growth_mean(inputs = [Fundamentals.current_liabilities], mask = base_universe)
# f3 = Var_growth_mean(inputs = [Fundamentals.financial_assets], mask = base_universe)
f41 = Var_growth_mean(inputs = [Fundamentals.payables], mask = base_universe)
# f5 = Var_growth_mean(inputs = [Fundamentals.restricted_cash], mask = base_universe)
f61 = Var_growth_mean(inputs = [Fundamentals.cap_ex_reported], mask = base_universe)
# f7 = Var_growth_mean(inputs = [Fundamentals.change_in_accrued_investment_income], mask = base_universe)
# f8 = Var_growth_mean(inputs = [Fundamentals.change_in_receivables], mask = base_universe)
# f9 = Var_growth_mean(inputs = [Fundamentals.change_in_working_capital], mask = base_universe)
# f10 = Var_growth_mean(inputs = [Fundamentals.changes_in_account_receivables], mask = base_universe)
# f11 = Var_growth_mean(inputs = [Fundamentals.domestic_sales], mask = base_universe)
# f12 = Var_growth_mean(inputs = [Fundamentals.sale_of_investment], mask = base_universe)
# f13 = Var_growth_mean(inputs = [Fundamentals.administrative_expense], mask = base_universe)
f141 = Var_growth_mean(inputs = [Fundamentals.net_income_extraordinary], mask = base_universe)
f151 = Var_growth_mean(inputs = [Fundamentals.pretax_income], mask = base_universe)
# f16 = Var_growth_mean(inputs = [Fundamentals.rent_and_landing_fees], mask = base_universe)
# f17 = Var_growth_mean(inputs = [Fundamentals.selling_and_marketing_expense], mask = base_universe)
f181 = Var_growth_mean(inputs = [Fundamentals.total_expenses], mask = base_universe)
# f19 = Var_growth_mean(inputs = [Fundamentals.total_revenue], mask = base_universe)
# f20 = Var_growth_mean(inputs = [Fundamentals.diluted_eps_growth], mask = base_universe)
# f21 = Var_growth_mean(inputs = [Fundamentals.days_in_payment], mask = base_universe)
# f22 = Var_growth_mean(inputs = [Fundamentals.days_in_sales], mask = base_universe)
f231 = Var_growth_mean(inputs = [Fundamentals.change_in_dividend_payable], mask = base_universe)


#Factor - 2 
# [5] - but almost no data
# f1 = Var_growth_q0(inputs = [Fundamentals.equity_investments], mask = base_universe)
# f2 = Var_growth_q0(inputs = [Fundamentals.securities_and_investments], mask = base_universe)
# f3 = Var_growth_q0(inputs = [Fundamentals.trading_liabilities], mask = base_universe)
# f4 = Var_growth_q0(inputs = [Fundamentals.cash_flow_from_continuing_financing_activities], mask = base_universe)
f52 = Var_growth_q0(inputs = [Fundamentals.interest_received_direct], mask = base_universe)
# f6 = Var_growth_q0(inputs = [Fundamentals.purchase_of_investment], mask = base_universe)
# f7 = Var_growth_q0(inputs = [Fundamentals.interest_income_from_loans], mask = base_universe)
# f8 = Var_growth_q0(inputs = [Fundamentals.cash_conversion_cycle], mask = base_universe)
# f9 = Var_growth_q0(inputs = [Fundamentals.long_term_debt_equity_ratio], mask = base_universe)
# f10 = Var_growth_q0(inputs = [Fundamentals.long_term_debt_total_capital_ratio], mask = base_universe)

#Factor - 3
# f1 = Var_growth_stability(inputs = [Fundamentals.operating_income], mask = base_universe)
# f2 = Var_growth_stability(inputs = [Fundamentals.financial_assets], mask = base_universe)
# f3 = Var_growth_stability(inputs = [Fundamentals.interest_payable], mask = base_universe)
# f4 = Var_growth_stability(inputs = [Fundamentals.classesof_cash_payments], mask = base_universe)
# f5 = Var_growth_stability(inputs = [Fundamentals.gain_loss_on_investment_securities], mask = base_universe)
# f6 = Var_growth_stability(inputs = [Fundamentals.interest_received_cfo], mask = base_universe)
# f7 = Var_growth_stability(inputs = [Fundamentals.net_investment_purchase_and_sale], mask = base_universe)
# f8 = Var_growth_stability(inputs = [Fundamentals.receiptsfrom_customers], mask = base_universe)
# f9 = Var_growth_stability(inputs = [Fundamentals.gain_on_sale_of_business], mask = base_universe)
# f10 = Var_growth_stability(inputs = [Fundamentals.net_income_discontinuous_operations], mask = base_universe)
# f11 = Var_growth_stability(inputs = [Fundamentals.days_in_inventory], mask = base_universe)
# f12 = Var_growth_stability(inputs = [Fundamentals.days_in_payment], mask = base_universe)
# f13 = Var_growth_stability(inputs = [Fundamentals.days_in_sales], mask = base_universe)
# f14 = Var_growth_stability(inputs = [Fundamentals.long_term_debt_equity_ratio], mask = base_universe)
# f15 = Var_growth_stability(inputs = [Fundamentals.long_term_debt_total_capital_ratio], mask = base_universe)

#Factors - 4
# [14]
# f1 =  Var_to_TA_mean(inputs=[Fundamentals.financial_assets, Fundamentals.total_assets], mask = base_universe) 
# f2 =  Var_to_TA_mean(inputs=[Fundamentals.payables, Fundamentals.total_assets], mask = base_universe) 
# f3 =  Var_to_TA_mean(inputs=[Fundamentals.restricted_cash, Fundamentals.total_assets], mask = base_universe) 
# f4 =  Var_to_TA_mean(inputs=[Fundamentals.tangible_book_value, Fundamentals.total_assets], mask = base_universe) 
# f5 =  Var_to_TA_mean(inputs=[Fundamentals.cap_ex_reported, Fundamentals.total_assets], mask = base_universe) 
# f6 =  Var_to_TA_mean(inputs=[Fundamentals.cash_flow_from_continuing_financing_activities, Fundamentals.total_assets], mask = base_universe) 
# f7 =  Var_to_TA_mean(inputs=[Fundamentals.change_in_account_payable, Fundamentals.total_assets], mask = base_universe) 
# f8 =  Var_to_TA_mean(inputs=[Fundamentals.change_in_interest_payable, Fundamentals.total_assets], mask = base_universe) 
# f9 =  Var_to_TA_mean(inputs=[Fundamentals.change_in_payable, Fundamentals.total_assets], mask = base_universe) 
# f10 =  Var_to_TA_mean(inputs=[Fundamentals.foreign_sales, Fundamentals.total_assets], mask = base_universe) 
# f11 =  Var_to_TA_mean(inputs=[Fundamentals.investing_cash_flow, Fundamentals.total_assets], mask = base_universe) 
# f12 =  Var_to_TA_mean(inputs=[Fundamentals.net_investment_purchase_and_sale, Fundamentals.total_assets], mask = base_universe) 
# f13 =  Var_to_TA_mean(inputs=[Fundamentals.net_issuance_payments_of_debt, Fundamentals.total_assets], mask = base_universe) 
f144 =  Var_to_TA_mean(inputs=[Fundamentals.purchase_of_intangibles, Fundamentals.total_assets], mask = base_universe) 
# f15 =  Var_to_TA_mean(inputs=[Fundamentals.gainon_saleof_loans, Fundamentals.total_assets], mask = base_universe) 
# f16 =  Var_to_TA_mean(inputs=[Fundamentals.minority_interests, Fundamentals.total_assets], mask = base_universe) 
# f17 =  Var_to_TA_mean(inputs=[Fundamentals.current_ratio, Fundamentals.total_assets], mask = base_universe) 
# f18 =  Var_to_TA_mean(inputs=[Fundamentals.ebit_margin, Fundamentals.total_assets], mask = base_universe) 
# f19 =  Var_to_TA_mean(inputs=[Fundamentals.quick_ratio, Fundamentals.total_assets], mask = base_universe) 

#Factors - 5
# [9,13]
# f1 =  Var_to_TA_stability(inputs=[Fundamentals.trading_assets, Fundamentals.total_assets], mask = base_universe) 
# f2 =  Var_to_TA_stability(inputs=[Fundamentals.line_of_credit, Fundamentals.total_assets], mask = base_universe) 
# f3 =  Var_to_TA_stability(inputs=[Fundamentals.total_partnership_capital, Fundamentals.total_assets], mask = base_universe) 
# f4 =  Var_to_TA_stability(inputs=[Fundamentals.change_in_interest_payable, Fundamentals.total_assets], mask = base_universe) 
# f5 =  Var_to_TA_stability(inputs=[Fundamentals.changes_in_account_receivables, Fundamentals.total_assets], mask = base_universe) 
# f6 =  Var_to_TA_stability(inputs=[Fundamentals.changes_in_cash, Fundamentals.total_assets], mask = base_universe) 
# f7 =  Var_to_TA_stability(inputs=[Fundamentals.interest_paid_direct, Fundamentals.total_assets], mask = base_universe) 
# f8 =  Var_to_TA_stability(inputs=[Fundamentals.long_term_debt_payments, Fundamentals.total_assets], mask = base_universe) 
f95 =  Var_to_TA_stability(inputs=[Fundamentals.cost_of_revenue, Fundamentals.total_assets], mask = base_universe) 
# f10 =  Var_to_TA_stability(inputs=[Fundamentals.gain_on_sale_of_security, Fundamentals.total_assets], mask = base_universe) 
# f11 =  Var_to_TA_stability(inputs=[Fundamentals.interest_income_from_securities, Fundamentals.total_assets], mask = base_universe) 
# f12 =  Var_to_TA_stability(inputs=[Fundamentals.reconciled_cost_of_revenue, Fundamentals.total_assets], mask = base_universe) 
f135 =  Var_to_TA_stability(inputs=[Fundamentals.equity_per_share_growth, Fundamentals.total_assets], mask = base_universe) 
# f14 =  Var_to_TA_stability(inputs=[Fundamentals.cash_conversion_cycle, Fundamentals.total_assets], mask = base_universe) 
# f15 =  Var_to_TA_stability(inputs=[Fundamentals.operation_revenue_growth3_month_avg, Fundamentals.total_assets], mask = base_universe) 
# f16 =  Var_to_TA_stability(inputs=[Fundamentals.revenue_growth, Fundamentals.total_assets], mask = base_universe) 


#Factors - 6 
# [-3], [8] [8-3]
# f1 =  Var_to_TL_mean(inputs=[Fundamentals.interest_payable, Fundamentals.total_debt], mask = base_universe) 
# f2 =  Var_to_TL_mean(inputs=[Fundamentals.long_term_investments, Fundamentals.total_debt], mask = base_universe) 
f36 =  Var_to_TL_mean(inputs=[Fundamentals.cash_dividends_paid, Fundamentals.total_debt], mask = base_universe) 
# f4 =  Var_to_TL_mean(inputs=[Fundamentals.change_in_accrued_expense, Fundamentals.total_debt], mask = base_universe) 
# f5 =  Var_to_TL_mean(inputs=[Fundamentals.change_in_receivables, Fundamentals.total_debt], mask = base_universe) 
# f6 =  Var_to_TL_mean(inputs=[Fundamentals.net_income_discontinuous_operations, Fundamentals.total_debt], mask = base_universe) 
# f7 =  Var_to_TL_mean(inputs=[Fundamentals.net_income_from_tax_loss_carryforward, Fundamentals.total_debt], mask = base_universe) 
f86 =  Var_to_TL_mean(inputs=[Fundamentals.dps_growth , Fundamentals.total_debt], mask = base_universe) 


#Factors - 7
#[-10,-15,-20] 
# f1 =  Var_to_TL_stability(inputs=[Fundamentals.total_deposits, Fundamentals.total_debt], mask = base_universe) 
# f2 =  Var_to_TL_stability(inputs=[Fundamentals.accounts_receivable, Fundamentals.total_debt], mask = base_universe) 
# f3 =  Var_to_TL_stability(inputs=[Fundamentals.cash, Fundamentals.total_debt], mask = base_universe) 
# f4 =  Var_to_TL_stability(inputs=[Fundamentals.dividends_payable, Fundamentals.total_debt], mask = base_universe) 
# f5 =  Var_to_TL_stability(inputs=[Fundamentals.financial_assets, Fundamentals.total_debt], mask = base_universe) 
# f6 =  Var_to_TL_stability(inputs=[Fundamentals.gross_loan, Fundamentals.total_debt], mask = base_universe) 
# f7 =  Var_to_TL_stability(inputs=[Fundamentals.payables, Fundamentals.total_debt], mask = base_universe) 
# f8 =  Var_to_TL_stability(inputs=[Fundamentals.cash_flow_from_continuing_financing_activities, Fundamentals.total_debt], mask = base_universe) 
# f9 =  Var_to_TL_stability(inputs=[Fundamentals.cash_flow_from_continuing_investing_activities, Fundamentals.total_debt], mask = base_universe) 
f107 = Var_to_TL_stability(inputs=[Fundamentals.classesof_cash_payments, Fundamentals.total_debt], mask = base_universe) 
# f11 = Var_to_TL_stability(inputs=[Fundamentals.common_stock_dividend_paid, Fundamentals.total_debt], mask = base_universe) 
# f12 = Var_to_TL_stability(inputs=[Fundamentals.foreign_sales, Fundamentals.total_debt], mask = base_universe) 
# f13 = Var_to_TL_stability(inputs=[Fundamentals.gain_loss_on_investment_securities, Fundamentals.total_debt], mask = base_universe) 
# f14 = Var_to_TL_stability(inputs=[Fundamentals.gain_loss_on_sale_of_ppe, Fundamentals.total_debt], mask = base_universe) 
f157 = Var_to_TL_stability(inputs=[Fundamentals.increase_decrease_in_deposit, Fundamentals.total_debt], mask = base_universe) 
# f16 = Var_to_TL_stability(inputs=[Fundamentals.interest_received_direct, Fundamentals.total_debt], mask = base_universe) 
# f17 = Var_to_TL_stability(inputs=[Fundamentals.interest_income, Fundamentals.total_debt], mask = base_universe) 
# f18 = Var_to_TL_stability(inputs=[Fundamentals.net_income_from_tax_loss_carryforward, Fundamentals.total_debt], mask = base_universe) 
# f19 = Var_to_TL_stability(inputs=[Fundamentals.non_interest_income, Fundamentals.total_debt], mask = base_universe) 
f207 = Var_to_TL_stability(inputs=[Fundamentals.research_and_development, Fundamentals.total_debt], mask = base_universe) 
# f21 = Var_to_TL_stability(inputs=[Fundamentals.salaries_and_wages, Fundamentals.total_debt], mask = base_universe) 
# f22 = Var_to_TL_stability(inputs=[Fundamentals.securities_activities, Fundamentals.total_debt], mask = base_universe) 
# f23 = Var_to_TL_stability(inputs=[Fundamentals.net_income_cont_ops_growth, Fundamentals.total_debt], mask = base_universe) 
# f24 = Var_to_TL_stability(inputs=[Fundamentals.net_income_growth, Fundamentals.total_debt], mask = base_universe) 
# f25 = Var_to_TL_stability(inputs=[Fundamentals.operation_income_growth, Fundamentals.total_debt], mask = base_universe) 
# f26 = Var_to_TL_stability(inputs=[Fundamentals.quick_ratio, Fundamentals.total_debt], mask = base_universe) 


#Factor - 8
# [6,7,9]
# f1 =  Var_to_TC_mean(inputs=[Fundamentals.total_equity, Fundamentals.total_capitalization], mask = base_universe) 
# f2 =  Var_to_TC_mean(inputs=[Fundamentals.interest_payable, Fundamentals.total_capitalization], mask = base_universe) 
# f3 =  Var_to_TC_mean(inputs=[Fundamentals.invested_capital, Fundamentals.total_capitalization], mask = base_universe) 
# f4 =  Var_to_TC_mean(inputs=[Fundamentals.line_of_credit, Fundamentals.total_capitalization], mask = base_universe) 
# f5 =  Var_to_TC_mean(inputs=[Fundamentals.trading_liabilities, Fundamentals.total_capitalization], mask = base_universe) 
f68 =  Var_to_TC_mean(inputs=[Fundamentals.change_in_account_payable, Fundamentals.total_capitalization], mask = base_universe) 
f78 =  Var_to_TC_mean(inputs=[Fundamentals.change_in_restricted_cash, Fundamentals.total_capitalization], mask = base_universe) 
# f8 =  Var_to_TC_mean(inputs=[Fundamentals.net_investment_purchase_and_sale, Fundamentals.total_capitalization], mask = base_universe) 
f98 =  Var_to_TC_mean(inputs=[Fundamentals.sale_of_investment, Fundamentals.total_capitalization], mask = base_universe) 
# f10 =  Var_to_TC_mean(inputs=[Fundamentals.gainon_saleof_loans, Fundamentals.total_capitalization], mask = base_universe) 
# f11 =  Var_to_TC_mean(inputs=[Fundamentals.interest_income_from_leases, Fundamentals.total_capitalization], mask = base_universe) 
# f12 =  Var_to_TC_mean(inputs=[Fundamentals.fix_assets_turonver, Fundamentals.total_capitalization], mask = base_universe) 


#Factors - 9
[4]
# f1 =  Var_to_TC_stability(inputs=[Fundamentals.receivables, Fundamentals.total_capitalization], mask = base_universe) 
# f2 =  Var_to_TC_stability(inputs=[Fundamentals.trading_assets , Fundamentals.total_capitalization], mask = base_universe) 
# f3 =  Var_to_TC_stability(inputs=[Fundamentals.change_in_receivables, Fundamentals.total_capitalization], mask = base_universe) 
f49 =  Var_to_TC_stability(inputs=[Fundamentals.gain_loss_on_sale_of_business, Fundamentals.total_capitalization], mask = base_universe) 
# f5 =  Var_to_TC_stability(inputs=[Fundamentals.salaries_and_wages, Fundamentals.total_capitalization], mask = base_universe) 

#Factors - 10
# [1,-3]
f110 =  Var_to_EV_mean(inputs=[Fundamentals.capital_expenditure, Fundamentals.enterprise_value], mask = base_universe) 
# f2 =  Var_to_EV_mean(inputs=[Fundamentals.common_stock_payments, Fundamentals.enterprise_value], mask = base_universe) 
f310 =  Var_to_EV_mean(inputs=[Fundamentals.repurchase_of_capital_stock, Fundamentals.enterprise_value], mask = base_universe) 
# f4 =  Var_to_EV_mean(inputs=[Fundamentals.gainon_saleof_loans, Fundamentals.enterprise_value], mask = base_universe) 
# f5 =  Var_to_EV_mean(inputs=[Fundamentals.interest_income_from_leases, Fundamentals.enterprise_value], mask = base_universe) 
# f6 =  Var_to_EV_mean(inputs=[Fundamentals.receivable_turnover, Fundamentals.enterprise_value], mask = base_universe) 


#Factors - 11
# [1,3]
f111 =  Var_to_EV_stability(inputs=[Fundamentals.free_cash_flow, Fundamentals.enterprise_value], mask = base_universe) 
# f2 =  Var_to_EV_stability(inputs=[Fundamentals.gain_loss_on_investment_securities, Fundamentals.enterprise_value], mask = base_universe) 
f311 =  Var_to_EV_stability(inputs=[Fundamentals.gain_loss_on_sale_of_business, Fundamentals.enterprise_value], mask = base_universe) 
# f4 =  Var_to_EV_stability(inputs=[Fundamentals.unrealized_gain_loss_on_investment_securities, Fundamentals.enterprise_value], mask = base_universe) 

# Factors - 12
# [-7,10]
# f1 =  Var_to_TE_mean(inputs=[Fundamentals.cash, Fundamentals.total_equity], mask = base_universe) 
# f2 =  Var_to_TE_mean(inputs=[Fundamentals.current_assets, Fundamentals.total_equity], mask = base_universe) 
# f3 =  Var_to_TE_mean(inputs=[Fundamentals.retained_earnings, Fundamentals.total_equity], mask = base_universe) 
# f4 =  Var_to_TE_mean(inputs=[Fundamentals.securities_and_investments, Fundamentals.total_equity], mask = base_universe) 
# f5 =  Var_to_TE_mean(inputs=[Fundamentals.working_capital, Fundamentals.total_equity], mask = base_universe) 
# f6 =  Var_to_TE_mean(inputs=[Fundamentals.change_in_accrued_expense, Fundamentals.total_equity], mask = base_universe) 
f712 =  Var_to_TE_mean(inputs=[Fundamentals.changes_in_account_receivables, Fundamentals.total_equity], mask = base_universe) 
# f8 =  Var_to_TE_mean(inputs=[Fundamentals.dividend_paid_cfo , Fundamentals.total_equity], mask = base_universe) 
# f9 =  Var_to_TE_mean(inputs=[Fundamentals.earnings_losses_from_equity_investments, Fundamentals.total_equity], mask = base_universe) 
f1012 =  Var_to_TE_mean(inputs=[Fundamentals.free_cash_flow, Fundamentals.total_equity], mask = base_universe) 
# f11 =  Var_to_TE_mean(inputs=[Fundamentals.gainon_saleof_loans, Fundamentals.total_equity], mask = base_universe) 
# f12 =  Var_to_TE_mean(inputs=[Fundamentals.equity_per_share_growth, Fundamentals.total_equity], mask = base_universe) 
# f13 =  Var_to_TE_mean(inputs=[Fundamentals.debtto_assets, Fundamentals.total_equity], mask = base_universe) 
# f14 =  Var_to_TE_mean(inputs=[Fundamentals.interest_coverage, Fundamentals.total_equity], mask = base_universe) 
# f15 =  Var_to_TE_mean(inputs=[Fundamentals.operation_income_growth, Fundamentals.total_equity], mask = base_universe) 
# f16 =  Var_to_TE_mean(inputs=[Fundamentals.operation_margin, Fundamentals.total_equity], mask = base_universe) 


#Factors - 13
# NONE
# f1 =  Var_to_TE_stability(inputs=[Fundamentals.change_in_inventory, Fundamentals.total_equity], mask = base_universe) 
# f2 =  Var_to_TE_stability(inputs=[Fundamentals.change_in_working_capital, Fundamentals.total_equity], mask = base_universe) 
# f3 =  Var_to_TE_stability(inputs=[Fundamentals.common_stock_dividend_paid, Fundamentals.total_equity], mask = base_universe) 
# f4 =  Var_to_TE_stability(inputs=[Fundamentals.long_term_debt_issuance, Fundamentals.total_equity], mask = base_universe) 
# f5 =  Var_to_TE_stability(inputs=[Fundamentals.net_issuance_payments_of_debt, Fundamentals.total_equity], mask = base_universe) 
# f6 =  Var_to_TE_stability(inputs=[Fundamentals.fee_revenue_and_other_income, Fundamentals.total_equity], mask = base_universe) 
# f7 =  Var_to_TE_stability(inputs=[Fundamentals.fees_and_commissions, Fundamentals.total_equity], mask = base_universe) 
# f8 =  Var_to_TE_stability(inputs=[Fundamentals.interest_income_from_securities, Fundamentals.total_equity], mask = base_universe) 
# f9 =  Var_to_TE_stability(inputs=[Fundamentals.operating_revenue , Fundamentals.total_equity], mask = base_universe) 
# f10 = Var_to_TE_stability(inputs=[Fundamentals.total_revenue, Fundamentals.total_equity], mask = base_universe) 
# f11 = Var_to_TE_stability(inputs=[Fundamentals.net_income_cont_ops_growth, Fundamentals.total_equity], mask = base_universe) 
# f12 = Var_to_TE_stability(inputs=[Fundamentals.net_income_growth, Fundamentals.total_equity], mask = base_universe) 


# [110, 49, 311, 1314, 814, 518, 1315]

#Factor-14
# [8,13]
# f1 =  Var_to_TR_mean(inputs=[Fundamentals.accounts_payable, Fundamentals.total_revenue], mask = base_universe) 
# f2 =  Var_to_TR_mean(inputs=[Fundamentals.current_debt, Fundamentals.total_revenue], mask = base_universe) 
# f3 =  Var_to_TR_mean(inputs=[Fundamentals.equity_investments, Fundamentals.total_revenue], mask = base_universe) 
# f4 =  Var_to_TR_mean(inputs=[Fundamentals.line_of_credit, Fundamentals.total_revenue], mask = base_universe) 
# f5 =  Var_to_TR_mean(inputs=[Fundamentals.net_debt, Fundamentals.total_revenue], mask = base_universe) 
# f6 =  Var_to_TR_mean(inputs=[Fundamentals.payables, Fundamentals.total_revenue], mask = base_universe) 
# f7 =  Var_to_TR_mean(inputs=[Fundamentals.working_capital, Fundamentals.total_revenue], mask = base_universe) 
f814 =  Var_to_TR_mean(inputs=[Fundamentals.capital_expenditure, Fundamentals.total_revenue], mask = base_universe) 
# f9 =  Var_to_TR_mean(inputs=[Fundamentals.change_in_restricted_cash, Fundamentals.total_revenue], mask = base_universe) 
# f10 =  Var_to_TR_mean(inputs=[Fundamentals.net_business_purchase_and_sale, Fundamentals.total_revenue], mask = base_universe) 
# f11 =  Var_to_TR_mean(inputs=[Fundamentals.purchase_of_intangibles, Fundamentals.total_revenue], mask = base_universe) 
# f12 =  Var_to_TR_mean(inputs=[Fundamentals.unrealized_gain_loss_on_investment_securities, Fundamentals.total_revenue], mask = base_universe) 
f1314 =  Var_to_TR_mean(inputs=[Fundamentals.earningsfrom_equity_interest_net_of_tax, Fundamentals.total_revenue], mask = base_universe) 
# f14 =  Var_to_TR_mean(inputs=[Fundamentals.gain_losson_saleof_assets, Fundamentals.total_revenue], mask = base_universe) 
# f15 =  Var_to_TR_mean(inputs=[Fundamentals.gainon_saleof_loans, Fundamentals.total_revenue], mask = base_universe) 
# f16 =  Var_to_TR_mean(inputs=[Fundamentals.net_income_extraordinary, Fundamentals.total_revenue], mask = base_universe) 
# f17 =  Var_to_TR_mean(inputs=[Fundamentals.salaries_and_wages, Fundamentals.total_revenue], mask = base_universe) 
# f18 =  Var_to_TR_mean(inputs=[Fundamentals.equity_per_share_growth, Fundamentals.total_revenue], mask = base_universe) 
# f19 =  Var_to_TR_mean(inputs=[Fundamentals.interest_coverage, Fundamentals.total_revenue], mask = base_universe) 
# f20 =  Var_to_TR_mean(inputs=[Fundamentals.inventory_turnover, Fundamentals.total_revenue], mask = base_universe) 
# f21 =  Var_to_TR_mean(inputs=[Fundamentals.net_income_cont_ops_growth, Fundamentals.total_revenue], mask = base_universe) 
# f22 =  Var_to_TR_mean(inputs=[Fundamentals.net_income_growth, Fundamentals.total_revenue], mask = base_universe) 
# f23 =  Var_to_TR_mean(inputs=[Fundamentals.payment_turnover, Fundamentals.total_revenue], mask = base_universe) 
# f24 =  Var_to_TR_mean(inputs=[Fundamentals.receivable_turnover, Fundamentals.total_revenue], mask = base_universe) 

#Factors - 15
# [2,7,9,11,13,14]
# f1 =  Var_to_TR_stability(inputs=[Fundamentals.total_deposits, Fundamentals.total_revenue], mask = base_universe) 
f215 =  Var_to_TR_stability(inputs=[Fundamentals.total_equity, Fundamentals.total_revenue], mask = base_universe) 
# f3 =  Var_to_TR_stability(inputs=[Fundamentals.accrued_investment_income, Fundamentals.total_revenue], mask = base_universe) 
# f4 =  Var_to_TR_stability(inputs=[Fundamentals.current_assets, Fundamentals.total_revenue], mask = base_universe) 
# f5 =  Var_to_TR_stability(inputs=[Fundamentals.current_liabilities, Fundamentals.total_revenue], mask = base_universe) 
# f6 =  Var_to_TR_stability(inputs=[Fundamentals.receivables, Fundamentals.total_revenue], mask = base_universe) 
f715 =  Var_to_TR_stability(inputs=[Fundamentals.restricted_cash, Fundamentals.total_revenue], mask = base_universe) 
# f8 =  Var_to_TR_stability(inputs=[Fundamentals.earnings_losses_from_equity_investments, Fundamentals.total_revenue], mask = base_universe) 
f915 =  Var_to_TR_stability(inputs=[Fundamentals.gain_loss_on_sale_of_ppe, Fundamentals.total_revenue], mask = base_universe) 
# f10 =  Var_to_TR_stability(inputs=[Fundamentals.gain_loss_on_sale_of_business, Fundamentals.total_revenue], mask = base_universe) 
f1115 =  Var_to_TR_stability(inputs=[Fundamentals.operating_gains_losses, Fundamentals.total_revenue], mask = base_universe) 
# f12 =  Var_to_TR_stability(inputs=[Fundamentals.sale_of_investment, Fundamentals.total_revenue], mask = base_universe) 
f1315 =  Var_to_TR_stability(inputs=[Fundamentals.gross_profit, Fundamentals.total_revenue], mask = base_universe) 
f1415 =  Var_to_TR_stability(inputs=[Fundamentals.interest_income_from_loans, Fundamentals.total_revenue], mask = base_universe) 
# f15 =  Var_to_TR_stability(inputs=[Fundamentals.interest_income_from_securities, Fundamentals.total_revenue], mask = base_universe) 
# f16 =  Var_to_TR_stability(inputs=[Fundamentals.securities_activities, Fundamentals.total_revenue], mask = base_universe) 


#Factors - 16
# [1,11]
f116 =  Var_to_TInv_mean(inputs=[Fundamentals.accrued_investment_income, Fundamentals.total_investments], mask = base_universe) 
# f2 =  Var_to_TInv_mean(inputs=[Fundamentals.cash, Fundamentals.total_investments], mask = base_universe) 
# f3 =  Var_to_TInv_mean(inputs=[Fundamentals.current_debt, Fundamentals.total_investments], mask = base_universe) 
# f4 =  Var_to_TInv_mean(inputs=[Fundamentals.long_term_debt, Fundamentals.total_investments], mask = base_universe) 
# f5 =  Var_to_TInv_mean(inputs=[Fundamentals.net_debt, Fundamentals.total_investments], mask = base_universe) 
# f6 =  Var_to_TInv_mean(inputs=[Fundamentals.net_loan, Fundamentals.total_investments], mask = base_universe) 
# f7 =  Var_to_TInv_mean(inputs=[Fundamentals.securities_and_investments, Fundamentals.total_investments], mask = base_universe) 
# f8 =  Var_to_TInv_mean(inputs=[Fundamentals.change_in_receivables, Fundamentals.total_investments], mask = base_universe) 
# f9 =  Var_to_TInv_mean(inputs=[Fundamentals.net_income_from_continuing_operations, Fundamentals.total_investments], mask = base_universe) 
# f10 =  Var_to_TInv_mean(inputs=[Fundamentals.administrative_expense, Fundamentals.total_investments], mask = base_universe) 
f1116 =  Var_to_TInv_mean(inputs=[Fundamentals.gain_losson_saleof_assets, Fundamentals.total_investments], mask = base_universe) 
# f12 =  Var_to_TInv_mean(inputs=[Fundamentals.gain_on_sale_of_business, Fundamentals.total_investments], mask = base_universe) 
# f13 =  Var_to_TInv_mean(inputs=[Fundamentals.gain_on_sale_of_security, Fundamentals.total_investments], mask = base_universe) 
# f14 =  Var_to_TInv_mean(inputs=[Fundamentals.interest_income, Fundamentals.total_investments], mask = base_universe) 
# f15 =  Var_to_TInv_mean(inputs=[Fundamentals.operating_expense, Fundamentals.total_investments], mask = base_universe) 
# f16 =  Var_to_TInv_mean(inputs=[Fundamentals.long_term_debt_equity_ratio, Fundamentals.total_investments], mask = base_universe) 
# f17 =  Var_to_TInv_mean(inputs=[Fundamentals.total_debt_equity_ratio, Fundamentals.total_investments], mask = base_universe) 



#Factors - 17
# [12]
# f1 =  Var_to_TInv_stability(inputs=[Fundamentals.gross_loan, Fundamentals.total_investments], mask = base_universe) 
# f2 =  Var_to_TInv_stability(inputs=[Fundamentals.receivables, Fundamentals.total_investments], mask = base_universe) 
# f3 =  Var_to_TInv_stability(inputs=[Fundamentals.trading_assets , Fundamentals.total_investments], mask = base_universe) 
# f4 =  Var_to_TInv_stability(inputs=[Fundamentals.cash_dividends_paid, Fundamentals.total_investments], mask = base_universe) 
# f5 =  Var_to_TInv_stability(inputs=[Fundamentals.change_in_restricted_cash, Fundamentals.total_investments], mask = base_universe) 
# f6 =  Var_to_TInv_stability(inputs=[Fundamentals.changes_in_account_receivables, Fundamentals.total_investments], mask = base_universe) 
# f7 =  Var_to_TInv_stability(inputs=[Fundamentals.investing_cash_flow, Fundamentals.total_investments], mask = base_universe) 
# f8 =  Var_to_TInv_stability(inputs=[Fundamentals.net_income_from_continuing_operations, Fundamentals.total_investments], mask = base_universe) 
# f9 =  Var_to_TInv_stability(inputs=[Fundamentals.net_investment_purchase_and_sale, Fundamentals.total_investments], mask = base_universe) 
# f10 =  Var_to_TInv_stability(inputs=[Fundamentals.operating_cash_flow, Fundamentals.total_investments], mask = base_universe) 
# f11 =  Var_to_TInv_stability(inputs=[Fundamentals.administrative_expense, Fundamentals.total_investments], mask = base_universe) 
f1217 =  Var_to_TInv_stability(inputs=[Fundamentals.earningsfrom_equity_interest_net_of_tax, Fundamentals.total_investments], mask = base_universe) 
# f13 =  Var_to_TInv_stability(inputs=[Fundamentals.ebit, Fundamentals.total_investments], mask = base_universe) 
# f14 =  Var_to_TInv_stability(inputs=[Fundamentals.ebitda, Fundamentals.total_investments], mask = base_universe) 
# f15 =  Var_to_TInv_stability(inputs=[Fundamentals.fees_and_commissions, Fundamentals.total_investments], mask = base_universe) 
# f16 =  Var_to_TInv_stability(inputs=[Fundamentals.net_income_income_statement, Fundamentals.total_investments], mask = base_universe) 
# f17 =  Var_to_TInv_stability(inputs=[Fundamentals.net_income_common_stockholders, Fundamentals.total_investments], mask = base_universe) 
# f18 =  Var_to_TInv_stability(inputs=[Fundamentals.net_income_continuous_operations, Fundamentals.total_investments], mask = base_universe) 
# f19 =  Var_to_TInv_stability(inputs=[Fundamentals.net_income_discontinuous_operations, Fundamentals.total_investments], mask = base_universe) 
# f20 =  Var_to_TInv_stability(inputs=[Fundamentals.net_income_including_noncontrolling_interests, Fundamentals.total_investments], mask = base_universe) 
# f21 =  Var_to_TInv_stability(inputs=[Fundamentals.normalized_income, Fundamentals.total_investments], mask = base_universe) 
# f22 =  Var_to_TInv_stability(inputs=[Fundamentals.operating_income, Fundamentals.total_investments], mask = base_universe) 
# f23 =  Var_to_TInv_stability(inputs=[Fundamentals.pretax_income , Fundamentals.total_investments], mask = base_universe) 
# f24 =  Var_to_TInv_stability(inputs=[Fundamentals.common_equity_to_assets, Fundamentals.total_investments], mask = base_universe) 
# f25 =  Var_to_TInv_stability(inputs=[Fundamentals.ebit_margin, Fundamentals.total_investments], mask = base_universe) 
# f26 =  Var_to_TInv_stability(inputs=[Fundamentals.ebitda_margin, Fundamentals.total_investments], mask = base_universe) 
# f27 =  Var_to_TInv_stability(inputs=[Fundamentals.financial_leverage, Fundamentals.total_investments], mask = base_universe) 
# f28 =  Var_to_TInv_stability(inputs=[Fundamentals.net_margin, Fundamentals.total_investments], mask = base_universe) 
# f29 =  Var_to_TInv_stability(inputs=[Fundamentals.normalized_net_profit_margin, Fundamentals.total_investments], mask = base_universe) 
# f30 =  Var_to_TInv_stability(inputs=[Fundamentals.operation_margin, Fundamentals.total_investments], mask = base_universe) 
# f31 =  Var_to_TInv_stability(inputs=[Fundamentals.pretax_margin, Fundamentals.total_investments], mask = base_universe) 
# f32 =  Var_to_TInv_stability(inputs=[Fundamentals.roe, Fundamentals.total_investments], mask = base_universe) 
# f33 =  Var_to_TInv_stability(inputs=[Fundamentals.sales_per_employee, Fundamentals.total_investments], mask = base_universe) 


# Factor - 18
[5]
# f1 =  Var_to_MCap_mean(inputs=[Fundamentals.net_business_purchase_and_sale, USEquityPricing.close, Fundamentals.shares_outstanding], mask = base_universe) 
# f2 =  Var_to_MCap_mean(inputs=[Fundamentals.sale_of_business, USEquityPricing.close, Fundamentals.shares_outstanding], mask = base_universe) 
# f3 =  Var_to_MCap_mean(inputs=[Fundamentals.gainon_saleof_loans, USEquityPricing.close, Fundamentals.shares_outstanding], mask = base_universe) 
# f4 =  Var_to_MCap_mean(inputs=[Fundamentals.minority_interests, USEquityPricing.close, Fundamentals.shares_outstanding], mask = base_universe) 
f518 =  Var_to_MCap_mean(inputs=[Fundamentals.non_interest_expense, USEquityPricing.close, Fundamentals.shares_outstanding], mask = base_universe) 
# f6 =  Var_to_MCap_mean(inputs=[Fundamentals.non_interest_income, USEquityPricing.close, Fundamentals.shares_outstanding], mask = base_universe) 
# f7 =  Var_to_MCap_mean(inputs=[Fundamentals.selling_and_marketing_expense, USEquityPricing.close, Fundamentals.shares_outstanding], mask = base_universe) 
# f8 =  Var_to_MCap_mean(inputs=[Fundamentals.operation_income_growth, USEquityPricing.close, Fundamentals.shares_outstanding], mask = base_universe) 
# f9 =  Var_to_MCap_mean(inputs=[Fundamentals.sales_per_employee, USEquityPricing.close, Fundamentals.shares_outstanding], mask = base_universe) 


# Factor - 19
# [1,5]
f119 =  Var_to_MCap_stability(inputs=[Fundamentals.operating_gains_losses, USEquityPricing.close, Fundamentals.shares_outstanding], mask = base_universe) 
# f2 =  Var_to_MCap_stability(inputs=[Fundamentals.repurchase_of_capital_stock, USEquityPricing.close, Fundamentals.shares_outstanding], mask = base_universe) 
# f3 =  Var_to_MCap_stability(inputs=[Fundamentals.gain_on_sale_of_security, USEquityPricing.close, Fundamentals.shares_outstanding], mask = base_universe) 
# f4 =  Var_to_MCap_stability(inputs=[Fundamentals.net_investment_income, USEquityPricing.close, Fundamentals.shares_outstanding], mask = base_universe) 
f519 =  Var_to_MCap_stability(inputs=[Fundamentals.diluted_eps_growth, USEquityPricing.close, Fundamentals.shares_outstanding], mask = base_universe) 
# f6 =  Var_to_MCap_stability(inputs=[Fundamentals.diluted_cont_eps_growth, USEquityPricing.close, Fundamentals.shares_outstanding], mask = base_universe) 
# f7 =  Var_to_MCap_stability(inputs=[Fundamentals.dps_growth , USEquityPricing.close, Fundamentals.shares_outstanding], mask = base_universe) 


# pipe_my_alpha_factor = Pipeline(
#     columns={

#         'f1': f1,
#         'f2': f2,
#         'f3': f3,
#         'f4': f4,
#         'f5': f5,
      
#         'f6': f6,
#         'f7': f7,
#         'f8': f8,
#         'f9': f9,
#         'f10': f10,
        
#         'f11': f11,
#         'f12': f12,
#         'f13': f13,
#         'f14': f14,
#         'f15': f15,

#         'f16': f16,
#         'f17': f17,
#         'f18': f18,
#         'f19': f19,
#         'f20': f20,

#         'f21': f21,
#         'f22': f22,
#         'f23': f23,
#         'f24': f24,
#         'f25': f25,
#         'f26': f26,

#         'f31': f31,
#         'f32': f32,
#         'f33': f33,

#         'vol': vol,
#         'log_mkt_cap': log_mkt_cap,
#         'mkt_cap':mkt_cap,
#         'industry': industry,
#         'sector': sector,
#         'industryGroup': industryGroup
#     },
#     screen=base_universe
# )


pipe_my_alpha_factor = Pipeline(
    columns={
         
        
        # [181, 144, 95, 86, 107, 157]
#         'f41':f41,
#         'f61':f61,
#         'f141':f141,
#         'f151':f151,
#         'f181':f181,
#         'f231':f231,  #NDW
        
#         'f52': f52,
#         'f144': f144,
#         'f95': f95,
#         'f135': f135,  #DW
#         'f36': f36, #DW
#         'f86': f86,
        
        
#         [78,49,110,311,814,1314]
#         'f107': f107,
#         'f157': f157,
#         'f207': f207,  #DW
#         'f68': f68,   #DW
#         'f78': f78, 
#         'f98': f98,   #DW
        

        'f49': f49,
        'f110': f110,  
#         'f310': f310, #DW
#         'f111': f111, #DW 
        'f311': f311, 
#         'f712': f712, #DW

#         'f1012': f1012,
        'f814': f814,
        'f1314': f1314,
       
              
#         'f215': f215,
#         'f715': f715,
#         'f915': f915,

#         'f1115': f1115,
        'f1315': f1315,
#         'f1415': f1415,
#         'f116': f116,
#         'f1116': f1116, 
#         'f1217': f1217, 

        'f518': f518,
#         'f119': f119,   #DW
#         'f519': f519, #DW
        
        'vol': vol,
        'log_mkt_cap': log_mkt_cap,
        'mkt_cap':mkt_cap,
        'industry': industry,
        'sector': sector,
        'industryGroup': industryGroup
    },
    screen=base_universe
)


output = run_pipeline(pipe_my_alpha_factor, sd, ed)
# print output.head(5)

In [ ]:
# print output[['f6','f7']].tail(20)

In [ ]:
#Factors1 = [4,6,14,15,18,23]
#factors2 = [5]
#factors4 = [14]
#Factors5 = [9,13] 
#Factors6 = [-3], [8] [8-3]
#Factor7 = [-10,-15,-20]
#Factor8 = [6,7,9]
#Factor9 = [4]
#Factor10 = [1,-3]
#Factor11 = [1,3]
#Factor12 = [-7,10]
#Factor13 = []
#Factor14 = [8,13]
#Factor15 = [2,7,9,11,13,14]
#Factor16 = [1,11]
#Factor17 = [12]
#Factor18 = [5]
#Factor19 = [1,5]


# factors = [1,2,3,4,5,6,7] 
# factors = [6,7,8,9,10,11]
# factors = [11,12,13,14,15]
# factors = [16,17,18,19,20]
# factors = [21,22,23,24,25]
# factors = [31,32,33]

# factors = [41,61,141,151,181]
# factors = [52,144,95,135,36,86]
# factors = [107,157,207,68,78,98]
# factors = [49,110,310,111,311,712]            
# factors = [1012, 814, 1314, 215, 715, 915]
# factors = [1115, 1315, 1415, 116, 1116, 1217]
# factors = [518, 119, 519]


#n
# factors = [231, 52, 144, 95, 86]
#Works - 1
# factors = [181, 144, 95, 86, 107, 157]
# factors = [49,110,311,814,1314]
# factors =  [215, 1315, 1415, 518]

factors = [110, 49, 311, 1314, 814, 518, 1315]
        

allnames = []
#Compute Weighted Factor value (within sector)
for x in factors:
    wfname = 'weightedFactor' + str(x)
    tfname = 'f'+str(x)
    output[wfname] = output[tfname]*output['mkt_cap']

    output['flag'] = 0
    output.loc[output[tfname].notnull(), 'flag'] = 1
    output['mkt_cap_'+tfname] = output['mkt_cap'] * output['flag']
    output[wfname] = output[wfname]*output['flag']
    
#     output = output.drop([tfname], axis=1)

    allnames.append(wfname)
    allnames.append('mkt_cap_'+tfname)
    

#Compute Sectoral Market Cap
# agg_sector_ = output.groupby([output.index.get_level_values(0),'sector'])[allnames].sum().reset_index()
# for x in factors:
#     output['flag'] = 0
#     tfname = 'f'+str(x)
#     wfname = 'weightedFactor' + str(x)
#     output.loc[output[tfname].notnull(), 'flag'] = 1
#     output['mkt_cap_'+tfname] = output['mkt_cap'] * output['flag']
#     output[wfname] = output[wfname]*output['flag']
    
agg_sector_ = output.groupby([output.index.get_level_values(0),'sector'])[allnames].sum().reset_index()

print agg_sector_.head(100)



In [ ]:
# print output[allnames].head(100)
# output.reset_index(inplace=True)
# output.drop(allnames, axis=1, inplace=True)

    output['sector'] = output['sector'].astype(str)
    output = pd.merge(output, df, on='sector', how='outer')



In [ ]:
#reset the output df
output = pd.DataFrame()

for x in factors:
    wfname = 'weightedFactor' + str(x)
    tfname = 'f'+str(x)
    agg_sector_[wfname] = agg_sector_[wfname]/agg_sector_['mkt_cap_'+tfname]
    agg_sector_.drop(['mkt_cap_'+tfname], axis=1, inplace=True)
    
# agg_sector_.rename(columns = {'mkt_cap_f':'sector_mkt_cap'}, inplace=True)
# agg_sector_.drop(['sector_mkt_cap'], axis=1, inplace=True)
agg_sector_['sector'] = agg_sector_['sector'].astype(str)
# print agg_sector_.head()

sector_code_sid = pd.DataFrame({
    'sector':['101.0','102.0','103.0','104.0','205.0','206.0','207.0','308.0','309.0','310.0','311.0'],
#     'sid':symbols(['XLB','XLY','XLF','IYR','XLP','XLV','XLU','XTL','XLE','XLI','XLK'])})
    'sid':symbols(['VAW', 'VCR', 'VFH', 'VNQ', 'VDC', 'VHT', 'VPU', 'VOX', 'VDE', 'VIS', 'VGT'])})

# print data.head()

In [ ]:
# print agg_sector_.head()

In [ ]:
# print sector_code_sid
data = pd.merge(agg_sector_, sector_code_sid, on='sector', how='outer').dropna()  
data.set_index(['level_0', 'sid'], inplace=True)

In [ ]:
# print data['alpha5'].head(100)

In [ ]:
# print data
for x in factors:
    wfname = 'weightedFactor' + str(x)
    tfname = 'f'+str(x)
    alphaname = 'alpha'+str(x)
    data[alphaname] = signalize(data[wfname])
    data[alphaname] = data[alphaname].sub(data[alphaname].mean(level=0), level=0)


In [ ]:
# We need to get a little more pricing data than the 
# length of our factor so we can compare forward returns.
# We'll tack on another month in this example.
assets = data.index.levels[1].unique()
pricing = get_pricing(assets, start_date=sd, end_date=ed, fields='close_price')
# pricing.head(10)

In [ ]:
returns = pricing.pct_change(periods=5).shift(-5)
#Demean the returns
returns = returns.sub(returns.mean(axis=1), axis=0).stack()
# print returns

In [ ]:
data = pd.concat([returns.rename('returns'), data], axis=1).dropna()
data.reset_index(inplace=True)
data = data.rename(columns={'level_0':'dates', 'level_1': 'sid'})

In [ ]:
#Alpha3, Alpha4, Alpha5, Alpha9 are highly correlated
# print data['alpha3'].corr(data['alpha6'])
# data['alpha3459'] = (data['alpha3'] + data['alpha4'] + data['alpha5'] + data['alpha9'])/4
# print data['alpha15']
# d = data[['alpha1','alpha2']][-3002:]
# print d

In [ ]:
#2014 -> 1,5 
import statsmodels.api as sm

# factor_names = [

#     'alpha1',
#     'alpha2',
#     'alpha3',
#     'alpha4',
#     'alpha5',
    
#     'alpha6',
#     'alpha7',
#     'alpha8',
#     'alpha9',
#     'alpha10',

#     'alpha11',
#     'alpha12',
#     'alpha13',
#     'alpha14',
#     'alpha15',

#     'alpha16',    
#     'alpha17',
#     'alpha18',
#     'alpha19',
#     'alpha20',

#     'alpha21',
#     'alpha22',
#     'alpha23',
#     'alpha24',
#     'alpha25',

#     'alpha26',

#     'alpha31',
#     'alpha32',
#     'alpha33',
    
# ]

# [181, 144, 95, 86, 107, 157]
factor_names = [
#     'alpha41',  #DW
#     'alpha61', #DW
#     'alpha141', #DW
#     'alpha151', @DW
#     'alpha181',  #W
#     'alpha231',  #DW
    
#     'alpha52',  #NDW
#     'alpha144', #W
#     'alpha95',  #W
#     'alpha135',  #DW
#     'alpha36',   #Dw
#     'alpha86', #W
    
#         'alpha107',  #W
#         'alpha157',   #W
#         'alpha207',   #Dw
#         'alpha68',   #DW
    
#         'alpha78', #DW
#         'alpha98',   #DW
        
    'alpha49',   #W
    'alpha110', #W
#     'alpha310',  #DW
#     'alpha111', #DW
    'alpha311', #W
#     'alpha712',  #DW
    
#     'alpha1012', #DW
    'alpha814', #W
    'alpha1314', #W
    
#     'alpha215',
#     'alpha715', #DW
#     'alpha915', #DW
    
#        'alpha1115', #Dw
        'alpha1315',  
#     'alpha1415',
#     'alpha116', #Dw
#     'alpha1116', #DW
#     'alpha1217' #Dw
    
    'alpha518',
#     'alpha119', #DW
#     'alpha519', #Dw
]

# data['final1'] = data['alpha3'] + data['alpha5'] + data['alpha6'] + data['alpha9'] + data['alpha11']
# data['final2'] = -data['alpha3'] + data['alpha8'] 

est = sm.OLS(
    100*data[['returns']], 
#     data[['final2']])
    data[factor_names])
est2 = est.fit(cov_type='cluster', cov_kwds={'groups': data['dates']})

print est2.summary()

## Run the Pipeline

In [ ]:
d = data.set_index(['dates', 'sid'])

# net_alpha = d['alpha4'] - d['alpha6']
# net_alpha = d['alpha4'] - d['alpha6'] - d['alpha14'] + d['alpha15'] + 
# net_alpha = d['alpha18']
# F1
# net_alpha = d['alpha4'] - d['alpha6'] - d['alpha14'] + d['alpha15'] + d['alpha18']
# net_alpha = 0.176*d['alpha4'] - 0.183*d['alpha6'] - 0.125*d['alpha14'] + 0.135*d['alpha15'] + 0.115*d['alpha18']


# net_alpha = -(d['alpha10'] + d['alpha15']) 
# net_alpha = -(d['alpha10'] + d['alpha15'] + d['alpha20'])

#12

# net_alpha = d['alpha41'] - d['alpha61'] - d['alpha141'] + d['alpha151'] + d['alpha181'] 
# net_alpha = d['alpha52'] + d['alpha144'] + d['alpha95'] + d['alpha86']
# net_alpha =      d['alpha107'] + \
#         d['alpha157'] + \
#         d['alpha207'] + \
#         d['alpha68'] + \
#         d['alpha78'] + \
#         d['alpha98'] 


# [49,110,310,111,311,712]
# net_alpha = d['alpha110'] + d['alpha311'] + d['alpha49']
# net_alpha = d['alpha814'] - d['alpha1314'] + d['alpha215'] 

# net_alpha =        d['alpha1115'] + \
#         d['alpha1315'] + \
#     d['alpha1415'] + \
#     d['alpha116'] + \
#     d['alpha1116'] + \
#     d['alpha1217']

# net_alpha = d['alpha1315'] + d['alpha1415'] 
# net_alpha = d['alpha116'] + d['alpha1116']
# net_alpha = d['alpha518'] 

# [231, 52, 144, 95, 86]
# net_alpha = d['alpha86'] + d['alpha144'] + d['alpha95']    #0.033/18.397/0.06/0.171/7.998

# net_alpha = d['alpha86'] + d['alpha144'] + d['alpha95'] + d['alpha181']  #0.041/20.272/0.056/0.169/7.902


# net_alpha = d['alpha86'] + d['alpha144'] + d['alpha95'] - d['alpha107']  #0.036/19.173/0.064/0.190/8.889

# net_alpha = d['alpha86'] + d['alpha144'] + d['alpha95'] - d['alpha157'] - d['alpha107'] 

# net_alpha = d['alpha86'] + d['alpha144'] + d['alpha95'] - d['alpha157'] - d['alpha107'] + d['alpha181']

# net_alpha = d['alpha49'] #0.01/11.46/0.026/0.079/3.7
# net_alpha = d['alpha110'] + d['alpha49'] #0.022/11.81/0.03/0.088/4.126
# net_alpha = d['alpha311']
# net_alpha = d['alpha110'] + d['alpha49'] + d['alpha311'] #0.020/14.958/0.037/0.109/5.102
# net_alpha = d['alpha814']
# net_alpha = d['alpha110'] + d['alpha49'] + d['alpha311'] + d['alpha814'] #0.024/12.720/0.036/0.106/4.981
# net_alpha = d['alpha1314']

#W net_alpha = d['alpha110'] + d['alpha49'] + d['alpha311'] - d['alpha1314']  #0.029/14.61/0.038/0.111/5.203

# net_alpha = d['alpha110'] + d['alpha49'] + d['alpha311'] - d['alpha1314']  + d['alpha814'] 

# net_alpha = d['alpha1315']  #0.031/13.874/0.03/0.088/4.1
# net_alpha = d['alpha518'] + d['alpha1315']  #0.04/10.381/0.024

net_alpha = d['alpha110'] + d['alpha49'] + d['alpha311'] - d['alpha1314'] + \
d['alpha814'] + d['alpha518'] + d['alpha1315']

# net_alpha = d['alpha1315']


factor_data = alphalens.utils.get_clean_factor_and_forward_returns(net_alpha,
                                                                   pricing,
                                                                   quantiles=5,
                                                                   periods=(5,10))

# alphalens.tears.create_returns_tear_sheet(factor_data)
# alphalens.tears.create_full_tear_sheet(factor_data)
alphalens.tears.create_summary_tear_sheet(factor_data)

### 